In [36]:
import os
import json
import cv2
import numpy as np
import albumentations as A

In [37]:
IMAGE_DIR = "../../data/sample-images"
OUTPUT_DIR = "../../data/sample-augmented"
ANNOTATION_FILE = "../../data/sample-COCO.json"
OUTPUT_JSON = "../../data/sample-augmented-COCO.json"
print(os.path.exists(IMAGE_DIR))  # Check if image exists
print(os.path.exists(OUTPUT_DIR)) 
print(os.path.exists(ANNOTATION_FILE))
print(os.path.exists(OUTPUT_JSON))

True
True
True
True


In [38]:
# Load COCO annotations
with open(ANNOTATION_FILE, "r") as f:
    coco_data = json.load(f)

In [39]:
# Create new dataset structure
new_coco_data = coco_data.copy()
new_coco_data["images"] = []
new_coco_data["annotations"] = []

In [40]:
# Image ID and annotation ID counters
image_id_counter = max(img["id"] for img in coco_data["images"]) + 1
annotation_id_counter = max(ann["id"] for ann in coco_data["annotations"]) + 1 # There can be multiple annotations for the same image

In [41]:
print(image_id_counter)
print(annotation_id_counter)

107
2413


In [42]:
# Define augmentation transformations
transforms = {
    "flipped": A.HorizontalFlip(p=1),
    "upsidedown": A.VerticalFlip(p=1),
    "flipped_upsidedown": A.Compose([A.HorizontalFlip(p=1), A.VerticalFlip(p=1)])
}

In [43]:
def update_annotation(ann, new_img_id, transform_func, img_width, img_height):
    """Update annotation (bounding boxes and segmentation) for a transformed image."""
    new_ann = ann.copy()
    new_ann["id"] = annotation_id_counter + new_ann["id"]
    new_ann["image_id"] = new_img_id

    # Transform bounding box (x, y, width, height)
    x, y, w, h = ann["bbox"]

    if transform_func == "flipped":
        x = img_width - (x + w)  # Flip X
    elif transform_func == "upsidedown":
        y = img_height - (y + h)  # Flip Y
    elif transform_func == "flipped_upsidedown":
        x = img_width - (x + w)
        y = img_height - (y + h)

    new_ann["bbox"] = [x, y, w, h]

    # Transform segmentation (if exists)
    if "segmentation" in ann and len(ann["segmentation"]) > 0:
        new_segmentation = []
        for segment in ann["segmentation"]:
            transformed_segment = []
            for i in range(0, len(segment), 2):
                px, py = segment[i], segment[i+1]
                if transform_func == "flipped":
                    px = img_width - px
                elif transform_func == "upsidedown":
                    py = img_height - py
                elif transform_func == "flipped_upsidedown":
                    px = img_width - px
                    py = img_height - py
                transformed_segment.extend([px, py])
            new_segmentation.append(transformed_segment)
        new_ann["segmentation"] = new_segmentation

    return new_ann

In [44]:
# Process each image
for img in coco_data["images"]:
    img_path = os.path.join(IMAGE_DIR, img["file_name"])
    image = cv2.imread(img_path)
    
    if image is None:
        continue

    print(f"Image: {img['file_name']}")

    height, width = image.shape[:2]

    # Copy original image and annotations
    new_coco_data["images"].append(img)
    new_coco_data["annotations"].extend([ann for ann in coco_data["annotations"] if ann["image_id"] == img["id"]])

    # Apply augmentations
    for transform_name, transform in transforms.items():
        transformed = transform(image=image)
        new_image = transformed["image"]

        # Generate new image file name
        new_filename = f"{os.path.splitext(img['file_name'])[0]}_{transform_name}.jpg"
        new_filepath = os.path.join(OUTPUT_DIR, new_filename)

        # Save augmented image
        cv2.imwrite(new_filepath, new_image)

        # Create new image entry in COCO
        new_image_entry = {
            "id": image_id_counter,
            "width": width,
            "height": height,
            "file_name": new_filename
        }
        new_coco_data["images"].append(new_image_entry)

        # Update annotations for new image
        for ann in coco_data["annotations"]:
            if ann["image_id"] == img["id"]:
                new_annotation = update_annotation(ann, image_id_counter, transform_name, width, height)
                new_coco_data["annotations"].append(new_annotation)

        image_id_counter += 1  # Increment image ID

# Save updated COCO JSON file
with open(OUTPUT_JSON, "w") as f:
    json.dump(new_coco_data, f, indent=4)

print("Data augmentation complete! Augmented images and updated JSON file saved.")

Image: 00010.png


[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00002.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00053.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00056.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00057.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00058.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.972] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00060.png'): can't open/read file: check file path/integrity
[ WARN:0@1105.973] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00061.png')

Image: 00009.png
Image: 00241.png
Data augmentation complete! Augmented images and updated JSON file saved.


[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00242.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00243.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00244.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00245.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00246.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00247.png'): can't open/read file: check file path/integrity
[ WARN:0@1106.368] global loadsave.cpp:241 findDecoder imread_('../../data/sample-images/00248.png')